In [100]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from clover import LocartSplit, RegressionScore
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV


Data Preparation and Preprocessing

In [101]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Load data
data = pd.read_csv("data/raw/bike/bike_train.csv")

# One-hot encode 'season' dan 'weather'
data = pd.concat([data, 
                  pd.get_dummies(data["season"], prefix="season"), 
                  pd.get_dummies(data["weather"], prefix="weather")], axis=1)
data.drop(["season", "weather"], axis=1, inplace=True)

# Ekstrak fitur waktu dari kolom datetime
data["hour"] = pd.to_datetime(data["datetime"]).dt.hour
data["day"] = pd.to_datetime(data["datetime"]).dt.dayofweek
data["month"] = pd.to_datetime(data["datetime"]).dt.month
data["year"] = pd.to_datetime(data["datetime"]).dt.year.map({2011: 0, 2012: 1})

# Hapus kolom yang tidak dipakai
data.drop(["datetime", "casual", "registered"], axis=1, inplace=True)

# Ganti nama kolom untuk fitur utama dan target
data = data.rename(columns={"temp": "x", "count": "y"})

# Split: train, calibration, validation
train_df, temp_df = train_test_split(data, test_size=0.4, random_state=1500)
cal_df, val_df = train_test_split(temp_df, test_size=0.5, random_state=1500)

# Info data
data.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10886 entries, 0 to 10885
Data columns (total 19 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   holiday     10886 non-null  int64  
 1   workingday  10886 non-null  int64  
 2   x           10886 non-null  float64
 3   atemp       10886 non-null  float64
 4   humidity    10886 non-null  int64  
 5   windspeed   10886 non-null  float64
 6   y           10886 non-null  int64  
 7   season_1    10886 non-null  bool   
 8   season_2    10886 non-null  bool   
 9   season_3    10886 non-null  bool   
 10  season_4    10886 non-null  bool   
 11  weather_1   10886 non-null  bool   
 12  weather_2   10886 non-null  bool   
 13  weather_3   10886 non-null  bool   
 14  weather_4   10886 non-null  bool   
 15  hour        10886 non-null  int32  
 16  day         10886 non-null  int32  
 17  month       10886 non-null  int32  
 18  year        10886 non-null  int64  
dtypes: bool(8), float64(3), i

Standarisasi Fitur

In [102]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from clover import LocartSplit, RegressionScore

# Standarisasi fitur
scaler = StandardScaler()
features = ['x', 'humidity', 'windspeed', 'hour', 'day', 'month', 'year']
X_train = scaler.fit_transform(train_df[features])
X_cal = scaler.transform(cal_df[features])
X_val = scaler.transform(val_df[features])

# Target
y_train = train_df['y']
y_cal = cal_df['y']

# Grid search untuk Random Forest
params = {
    'n_estimators': [100],
    'max_depth': [None, 5, 10]
}
gcv = GridSearchCV(RandomForestRegressor(random_state=1500), params, cv=5)
gcv.fit(X_train, y_train)
best_rf = gcv.best_estimator_

# LOCART
alpha = 0.1
locart = LocartSplit(RegressionScore, best_rf, alpha=alpha, is_fitted=True)
locart.fit(X_train, y_train)
locart.calib(X_cal, y_cal, random_seed=1500, cart_train_size=0.5)

# LOFOREST
loforest = LocartSplit(RegressionScore, best_rf, alpha=alpha, cart_type="RF", is_fitted=True)
loforest.fit(X_train, y_train)
loforest.calib(X_cal, y_cal, random_seed=1500, cart_train_size=0.5)

# Prediksi pada 500 data validasi
x_full = val_df[features].iloc[:500]
x_values = scaler.transform(x_full)
y_true = val_df.loc[x_full.index, 'y'].values

locart_values = locart.predict(x_values)
loforest_values = loforest.predict(x_values)


Definisi Fungsi Evaluasi: SMIS

In [104]:
def calculate_smis(y_true, lower, upper, alpha=0.1):
    """Calculate Scaled Mean Interval Score (SMIS) for prediction intervals, unscaled to match paper's hundreds output.
    
    Args:
        y_true: Array of true values.
        lower: Array of lower bounds of prediction intervals.
        upper: Array of upper bounds of prediction intervals.
        alpha: Miscoverage level (e.g., 0.1 for 90% coverage).
    
    Returns:
        smis: Unscaled Mean Interval Score (in hundreds).
    """
    mis = (upper - lower) + (2 / alpha) * (lower - y_true) * (y_true < lower) + \
          (2 / alpha) * (y_true - upper) * (y_true > upper)
    return np.mean(mis)

def calculate_ccad(y_true, lower, upper, y_pred, alpha=0.1, n_bins=10):
    """Calculate Conditional Coverage Absolute Deviation (CCAD) by binning.
    
    Args:
        y_true: Array of true values.
        lower: Array of lower bounds of prediction intervals.
        upper: Array of upper bounds of prediction intervals.
        y_pred: Array of predicted values (e.g., median or mean) for binning.
        alpha: Miscoverage level (e.g., 0.1 for 90% coverage).
        n_bins: Number of bins for grouping data.
    
    Returns:
        ccad: Conditional Coverage Absolute Deviation.
    """
    bins = pd.qcut(y_pred, q=n_bins, duplicates='drop')
    bin_indices = pd.Categorical(bins).codes
    unique_bins = np.unique(bin_indices[bin_indices != -1])
    
    deviations = []
    for bin_idx in unique_bins:
        mask = bin_indices == bin_idx
        if np.sum(mask) > 0:
            coverage = np.mean((y_true[mask] >= lower[mask]) & (y_true[mask] <= upper[mask]))
            deviation = np.abs(coverage - (1 - alpha))
            deviations.append(deviation)
    
    return np.mean(deviations) if deviations else 0.0

Evaluasi Model LOCART & LOFOREST

In [105]:
# Ground truth dari subset x_full
y_true = val_df.loc[x_full.index, 'y'].values

# Prediksi ulang terhadap x_full yang sudah distandarisasi
locart_val = locart.predict(x_values)
loforest_val = loforest.predict(x_values)

# Evaluasi LOCART
smis_locart = calculate_smis(y_true, locart_val[:, 0], locart_val[:, 1], alpha=alpha)
ccad_locart = calculate_ccad(y_true, locart_val[:, 0], locart_val[:, 1], y_pred_val, alpha=alpha)

# Evaluasi LOFOREST
smis_loforest = calculate_smis(y_true, loforest_val[:, 0], loforest_val[:, 1], alpha=alpha)
ccad_loforest = calculate_ccad(y_true, loforest_val[:, 0], loforest_val[:, 1], y_pred_val, alpha=alpha)


Hasil model LOCART & LOFOREST

In [106]:
print("=== SMIS and CCAD Results ===")
print(f"LOCART    -> SMIS: {smis_locart:.2f}, CCAD: {ccad_locart:.4f}")
print(f"LOFOREST  -> SMIS: {smis_loforest:.2f}, CCAD: {ccad_loforest:.4f}")


=== SMIS and CCAD Results ===
LOCART    -> SMIS: 191.07, CCAD: 0.0520
LOFOREST  -> SMIS: 192.50, CCAD: 0.0460


Perbandingan menggunakan model regresi standar

In [107]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.linear_model import LinearRegression, BayesianRidge
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.metrics import mean_squared_error

# === 1. Quantile Regression ===
q_lower = HistGradientBoostingRegressor(loss='quantile', quantile=alpha / 2,
                                        max_depth=2, max_iter=20, min_samples_leaf=1)
q_upper = HistGradientBoostingRegressor(loss='quantile', quantile=1 - alpha / 2,
                                        max_depth=2, max_iter=20, min_samples_leaf=1)
q_median = HistGradientBoostingRegressor(loss='squared_error',
                                         max_depth=2, max_iter=20, min_samples_leaf=1)

q_lower.fit(X_train, y_train)
q_upper.fit(X_train, y_train)
q_median.fit(X_train, y_train)

q_pred_lower = q_lower.predict(x_values)
q_pred_upper = q_upper.predict(x_values)
q_pred_median = q_median.predict(x_values)

# === 2. Linear Regression with Normality Assumption ===
linreg = LinearRegression()
linreg.fit(X_train, y_train)

y_pred_lin = linreg.predict(x_values)
y_train_pred_lin = linreg.predict(X_train)
resid_std = np.std(y_train - y_train_pred_lin)

z = 1.96  # Z value for 90% prediction interval
lin_lower = y_pred_lin - z * resid_std
lin_upper = y_pred_lin + z * resid_std

# === 3. Bayesian Ridge Regression ===
bayes = BayesianRidge()
bayes.fit(X_train, y_train)

y_pred_bayes, std_bayes = bayes.predict(x_values, return_std=True)
bayes_lower = y_pred_bayes - z * std_bayes
bayes_upper = y_pred_bayes + z * std_bayes

# === 4. Neural Network with MC Dropout ===
class MCDropoutNN(nn.Module):
    def __init__(self, input_dim):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, 128),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(64, 1)
        )

    def forward(self, x):
        return self.net(x)

# Buat tensor dari data
X_tensor = torch.tensor(X_train, dtype=torch.float32)
y_tensor = torch.tensor(y_train.values, dtype=torch.float32).view(-1, 1)

model = MCDropoutNN(input_dim=X_train.shape[1])
optimizer = torch.optim.Adam(model.parameters(), lr=0.005)  # learning rate diturunkan
loss_fn = nn.MSELoss()

# Training
model.train()
for epoch in range(1000):  # lebih lama, tapi lebih stabil
    optimizer.zero_grad()
    output = model(X_tensor)
    loss = loss_fn(output, y_tensor)
    loss.backward()
    optimizer.step()

# MC Dropout Sampling
model.eval()
x_tensor = torch.tensor(x_values, dtype=torch.float32)
mc_preds = []

with torch.no_grad():
    for _ in range(100):
        model.train()  # tetap aktifkan dropout
        preds = model(x_tensor).squeeze().numpy()
        mc_preds.append(preds)

mc_preds = np.stack(mc_preds)
nn_mean = mc_preds.mean(axis=0)
nn_std = mc_preds.std(axis=0)
nn_lower = nn_mean - z * nn_std
nn_upper = nn_mean + z * nn_std



In [ ]:
# Ambil ground truth y dari subset data validasi yang digunakan
y_true = val_df.loc[x_full.index, 'y'].values  

# Quantile Regression
smis_qr = calculate_smis(y_true, q_pred_lower, q_pred_upper, alpha=alpha)
ccad_qr = calculate_ccad(y_true, q_pred_lower, q_pred_upper, q_pred_median, alpha=alpha)

# Linear Regression
smis_lin = calculate_smis(y_true, lin_lower, lin_upper, alpha=alpha)
ccad_lin = calculate_ccad(y_true, lin_lower, lin_upper, y_pred_lin, alpha=alpha)

# Bayesian Regression
smis_bayes = calculate_smis(y_true, bayes_lower, bayes_upper, alpha=alpha)
ccad_bayes = calculate_ccad(y_true, bayes_lower, bayes_upper, y_pred_bayes, alpha=alpha)

# Neural Network MC Dropout
smis_nn = calculate_smis(y_true, nn_lower, nn_upper, alpha=alpha)
ccad_nn = calculate_ccad(y_true, nn_lower, nn_upper, nn_mean, alpha=alpha)


Hasil model regresi standar

In [110]:
results = pd.DataFrame({
    'Model': ['Quantile Regression', 'Linear Regression', 'Bayesian Regression', 'NN + MC Dropout'],
    'SMIS': [smis_qr, smis_lin, smis_bayes, smis_nn],
    'CCAD': [ccad_qr, ccad_lin, ccad_bayes, ccad_nn]
})

print(results.round(4))


                 Model      SMIS    CCAD
0  Quantile Regression  463.3678  0.0658
1    Linear Regression  641.7147  0.0580
2  Bayesian Regression  642.2072  0.0580
3      NN + MC Dropout  673.2267  0.2440


Hasil keseluruhan model

In [111]:
results = pd.DataFrame({
    'Model': [
        'LOCART', 'LOFOREST',
        'Quantile Regression', 'Linear Regression',
        'Bayesian Regression', 'NN + MC Dropout'
    ],
    'SMIS': [
        smis_locart, smis_loforest,
        smis_qr, smis_lin, smis_bayes, smis_nn
    ],
    'CCAD': [
        ccad_locart, ccad_loforest,
        ccad_qr, ccad_lin, ccad_bayes, ccad_nn
    ]
})

print(results.round(4))


                 Model      SMIS    CCAD
0               LOCART  191.0706  0.0520
1             LOFOREST  192.4979  0.0460
2  Quantile Regression  463.3678  0.0658
3    Linear Regression  641.7147  0.0580
4  Bayesian Regression  642.2072  0.0580
5      NN + MC Dropout  673.2267  0.2440
